# Results of Methodology

## Data Loading and Preparation

In [3]:
import multiprocessing as mp
from copy import copy
import itertools
import collections

import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, RandomForestRegressor, RandomForestClassifier

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, StratifiedKFold, GridSearchCV
from sklearn.model_selection._search import ParameterGrid

from xbcausalforest import XBCF

# Load model definitions from general module
# See https://github.com/johaupt/treatment-learn
#import sys
#sys.path.append('C:/Users/julia/OneDrive - Humboldt-Universitaet zu Berlin, CMS/Desktop_alt/thesis/code/treatment-learn')
import helper
from treatlearn.double_robust_transformation import DoubleRobustTransformer
from treatlearn.indirect import SingleModel, HurdleModel, TwoModelRegressor
from treatlearn.evaluation import transformed_outcome_loss

In [4]:
from helper import *

In [5]:
# For plotting in Latex font only
# import matplotlib
# import matplotlib.pyplot as plt
# from matplotlib import rc
# rc('font',**{'family':'serif','serif':['cm']})
# ## for Palatino and other serif fonts use:
# #rc('font',**{'family':'serif','serif':['Palatino']})
# rc('text', usetex=True)
# #matplotlib.rcParams['mathtext.fontset'] = 'cm'
# matplotlib.pyplot.title(r'ABC123 vs $\mathrm{ABC123}^{123}$')
# plt.rc('text.latex', preamble=r'\usepackage{underscore}')

In [6]:
from datetime import date

today = date.today()

Results of different runs, some after scaling

In [24]:
predictions = np.load(f"results/run_thorugh_2022-05-30.npy", allow_pickle=True)
predictions_test = [fold for fold in predictions]

In [7]:
# 5.2 All models, without scaling
DATA_PATH = "data/fashionB_clean_nonlinear.csv"
RESULT_PATH = "prediction_test_results_5.2"
#RESULT_PATH = "oracle_prediction_test_targeting" # for scaled predictions after selection
predictions = np.load(f"results/{RESULT_PATH}.npy", allow_pickle=True)
predictions_test = [fold for fold in predictions]

In [81]:
#5.3. Regularization: 50 folds with CATE Scaling
RESULT_PATH = "prediction_test_scaled_cv_CATE"
predictions_test = np.load(f"results/{RESULT_PATH}.npy", allow_pickle=True)
predictions_test = [fold for fold in predictions_test]
RESULT_PATH = "prediction_train_scaled_cv_CATE"
predictions_train = np.load(f"results/{RESULT_PATH}.npy", allow_pickle=True)
predictions_train = [fold for fold in predictions_train]

In [ ]:
#5.3. Regularization: 50 folds without CATE Scaling
RESULT_PATH = "prediction_test_cv"
predictions_test = np.load(f"results/{RESULT_PATH}.npy", allow_pickle=True)
predictions_test = [fold for fold in predictions_test]
RESULT_PATH = "prediction_train_cv"
predictions_train = np.load(f"results/{RESULT_PATH}.npy", allow_pickle=True)
predictions_train = [fold for fold in predictions_train]

In [23]:
#5.3. Regularization: 50 folds with Oracle Scaling, XBCF shifted beforehand
RESULT_PATH = "prediction_test_oracle_cv"
predictions_test = np.load(f"results/{RESULT_PATH}.npy", allow_pickle=True)
predictions_test = [fold for fold in predictions_test]
RESULT_PATH = "prediction_train_oracle_cv"
predictions_train = np.load(f"results/{RESULT_PATH}.npy", allow_pickle=True)
predictions_train = [fold for fold in predictions_train]


In [8]:
DATA_PATH = "data/fashionB_clean_nonlinear.csv"
SEED=42
N_SPLITS = 5
np.random.seed(SEED)

X = pd.read_csv(DATA_PATH)

DEBUG = False

# Load data

c = X.pop('converted').to_numpy()
g = X.pop('TREATMENT').to_numpy()
y = X.pop('checkoutAmount').to_numpy()
tau_conversion = X.pop('TREATMENT_EFFECT_CONVERSION')
tau_basket = X.pop('TREATMENT_EFFECT_BASKET')
tau_response = X.pop('TREATMENT_EFFECT_RESPONSE')


## Evaluate Conversion

In [9]:
eval_conversion = [calc_classification_error(outcome_dict["conversion"], y_true=c[outcome_dict["idx"]], g=np.nonzero(g[outcome_dict["idx"]]))
             for outcome_dict in predictions_test]

In [10]:
eval_conversion = pd.concat([pd.DataFrame(x) for x in eval_conversion], axis=0, keys=range(len(eval_conversion)))
eval_conversion.index.rename(["fold","metric"], inplace=True)

In [11]:
eval_conversion = eval_conversion.groupby("metric").mean().T

In [12]:
eval_conversion.index = pd.MultiIndex.from_tuples(eval_conversion.index.str.split("_", expand=True).tolist())
eval_conversion = eval_conversion.rename(mapper={"ROC-AUC": "ROC-AUC", "brier": "Brier Score"}, axis=1)

In [13]:
eval_conversion

metric                        ROC-AUC  Brier Score
single-model    outcome gbt  0.670755     0.100660
                hurdle  gbt  0.669654     0.100490
two-model       hurdle  gbt  0.657968     0.101391
Conversion-Rate              0.500000     0.105053

In [12]:
eval_conversion.to_latex(buf=f"results/conversion_prediction_quality_{today}_cv50.tex", float_format="%.3f")


## 5.1 Evaluate CATE

In [14]:
eval_test = [calc_prediction_error(outcome_dict["treatment_spending"],
                                   y[outcome_dict["idx"]], g[outcome_dict["idx"]], tau_true=tau_response[outcome_dict["idx"]],
                                   time_dict=outcome_dict['time_model']) #
             for outcome_dict in predictions_test]

In [15]:
eval_test

[{'single-model_outcome_gbt': {'transformed_outcome_loss': 4282.278213209041,
   'root_mean_squared_error': 2.9362007439922206,
   'mean_absolute_error': 2.158427412523232,
   'training_time': 25.900500297546387},
  'single-model_hurdle_gbt': {'transformed_outcome_loss': 4281.727286179715,
   'root_mean_squared_error': 2.8653531725174513,
   'mean_absolute_error': 2.0491341011279163,
   'training_time': 29.480369806289673},
  'two-model_outcome_gbt': {'transformed_outcome_loss': 4283.503663897647,
   'root_mean_squared_error': 3.205326271578667,
   'mean_absolute_error': 2.3491021007688273,
   'training_time': 20.190366506576538},
  'two-model_hurdle_gbt': {'transformed_outcome_loss': 4284.142360745319,
   'root_mean_squared_error': 3.2081669365743593,
   'mean_absolute_error': 2.4177086960835656,
   'training_time': 21.345075607299805},
  'dr_outcome_gbt': {'transformed_outcome_loss': 4283.651255448995,
   'root_mean_squared_error': 3.0025190259341152,
   'mean_absolute_error': 2.2802

In [16]:
eval_test_dataframe = pd.concat([pd.DataFrame(x) for x in eval_test], axis=0, keys=range(len(eval_test)))

In [17]:
eval_test_dataframe.index.rename(["fold","metric"], inplace=True)

In [18]:
eval_precision = eval_test_dataframe.groupby("metric").mean().T

In [19]:
eval_precision.index = pd.MultiIndex.from_tuples(eval_precision.index.str.split("_", expand=True).tolist())
eval_precision = eval_precision.rename(mapper={"transformed_outcome_loss": "TOL", "root_mean_squared_error": "RMSE", "mean_absolute_error": "MAE"}, axis=1)

ATE and Oracle are same as in paper. Rest is too high without tuning (also in comparison to ATE)

In [20]:
eval_precision.round(2)

metric                         TOL  RMSE   MAE  training_time
single-model outcome gbt   4179.48  3.06  2.24          25.88
             hurdle  gbt   4179.46  3.00  2.19          29.42
two-model    outcome gbt   4180.92  3.37  2.49          19.75
             hurdle  gbt   4182.10  3.34  2.52          21.85
dr           outcome gbt   4179.99  3.09  2.31          12.82
oracle                     4164.46  0.00  0.00            NaN
ATE                        4186.27  3.77  3.05            NaN
xbcf         outcome xbcf  4180.42  3.13  2.29        1319.67

In [19]:
# write out the table
eval_precision.to_latex(buf=f"results/treatment_prediction_quality_{today}.tex", float_format="%.2f")

In [20]:
import scipy.stats as stats
stats.spearmanr(eval_precision[["RMSE"]], eval_precision[["TOL"]])

SpearmanrResult(correlation=1.0, pvalue=0.0)

In [21]:
np.argsort(eval_precision[["RMSE"]].values.flatten())

array([4, 0, 2, 1, 3, 5], dtype=int64)

In [22]:
np.argsort(eval_precision[["TOL"]].values.flatten())


array([4, 0, 2, 1, 3, 5], dtype=int64)

## 5.2 Evaluate PIs
### PICP, MPIW

In [90]:
from helper import *
unc_eval_test = [helper.calc_uncertainty_metrics(outcome_dict["prediction_intervals"],
                                                 time_dict=outcome_dict["time_pi"],
                                                 tau_true=tau_response[outcome_dict["idx"]]
                                                 )
             for outcome_dict in predictions_test]

In [78]:
predictions_test

array([{'idx': array([    15,     17,     26, ..., 119238, 119241, 119243]), 'conversion': {'single-model_outcome_gbt': array([0.04322822, 0.0870595 , 0.24646043, ..., 0.09211538, 0.14967952,
              0.19753322]), 'single-model_hurdle_gbt': array([0.05122514, 0.09204595, 0.26105572, ..., 0.08445664, 0.13021363,
              0.19868886]), 'two-model_hurdle_gbt': array([0.07386939, 0.114136  , 0.19304165, ..., 0.10627231, 0.14940705,
              0.16250635]), 'Conversion-Rate__': array([0.11926778, 0.11926778, 0.11926778, ..., 0.11926778, 0.11926778,
              0.11926778])}, 'treatment_conversion': {'single-model_hurdle_gbt': array([0.01532561, 0.02987715, 0.05436079, ..., 0.04262516, 0.0550649 ,
              0.07573804]), 'two-model_hurdle_gbt': array([0.02977157, 0.04255592, 0.04795257, ..., 0.04866471, 0.07125806,
              0.06583043]), 'ATE__': array([0.04777018, 0.04777018, 0.04777018, ..., 0.04777018, 0.04777018,
              0.04777018]), 'oracle__': array([-0.

In [91]:
# change ordering of dicts
quantiles = [0.05, 0.32] #0.1,0.2,
unc_by_alpha = {}
for alpha in quantiles:
    unc_by_alpha[alpha] = []
    for folds in unc_eval_test:
        list = folds[alpha]
        unc_by_alpha[alpha].append(list)


array([0.87585312, 0.98491325, 0.92388781, ..., 1.00388629, 1.5186635 ,
       3.52860878])

In [93]:
unc_eval_test_dataframe_folds = {}
for alpha in quantiles:

    unc_eval_test_dataframe = pd.concat([pd.DataFrame(x) for x in unc_by_alpha[alpha]], axis=0, keys=range(len(eval_test)))
    unc_eval_test_dataframe.index.rename(["fold","metric"], inplace=True)
    unc_eval_test_dataframe.rename(str, axis='columns', inplace=True)
    unc_eval_test_dataframe_folds[alpha] = unc_eval_test_dataframe
    unc_eval_test_dataframe= unc_eval_test_dataframe.groupby("metric").mean().T
    print(unc_eval_test_dataframe.round(2))
    unc_eval_test_dataframe.to_latex(buf=f"figures/treatment_uncertainty_prediction_quality_{alpha}_{today}_scaled.tex", float_format="%.2f")

metric                 PICP  QNMPIW   Std  training_time
xbcf_outcome_xbcf      0.88    0.92  9.20          33.66
Agnostic_QR_two-model  0.97    1.23  3.45          28.64
CP_two-model_NN        1.00    2.23  3.88          81.05
MAPIE_two-model_naive  1.00    2.16  0.29           0.44
metric                 PICP  QNMPIW    Std  training_time
xbcf_outcome_xbcf      0.29    0.74   3.33          33.66
Agnostic_QR_two-model  0.79    7.12  20.81          25.08
CP_two-model_NN        0.99   19.01   0.00          71.80
MAPIE_two-model_naive  1.00   17.79   0.00           0.34


Evaluation Tables

In [ ]:
# remove instances
for alpha in quantiles:
    unc_eval_test_dataframe_folds[alpha] = unc_eval_test_dataframe_folds[alpha].loc[:, ~(unc_eval_test_dataframe_folds[alpha].columns.str.contains('single') | unc_eval_test_dataframe_folds[alpha].columns.str.contains('dr')
                                              | unc_eval_test_dataframe_folds[alpha].columns.str.contains('DR'))].groupby("metric").mean().T
    unc_eval_test_dataframe_folds[alpha]['training_time'] = unc_eval_test_dataframe_folds[alpha]['training_time']/60
    unc_eval_test_dataframe_folds[alpha].to_latex(buf=f"figures/treatment_uncertainty_prediction_quality_small_{alpha}_{today}.tex", float_format="%.2f")


In [82]:

df = pd.DataFrame(unc_eval_test_dataframe_folds[alpha].T)

fold                0                                                1  \
metric           PICP     QNMPIW           Std training_time      PICP   
Names                                                                    
QR_GBR       0.737851   6.670562  2.245883e+01      1.561073  0.726739   
Split-CP_RF  0.986960  22.591146  2.980303e-14      0.734264  0.981760   
CQR_RF_Sym   0.998952  14.581862  2.600921e+01     19.793254  0.998742   
MAPIE_Naive  0.999203  17.528388  1.503291e-14      0.021092  0.998658   
MAPIE_CV+    0.999916  18.798867  1.410176e+00      0.103277  0.999706   
XBCF         0.323619   0.768368  2.004063e+00     19.681213  0.295107   
Local-CP_RF  0.987085  16.420577  3.841837e+01    432.251704  0.988343   
Split-CP_NN  0.987589  16.004842  1.372450e-14     30.302477  0.996436   
Local-CP_NN  0.990901  11.820649  1.132737e+01     43.518175  0.996268   
CQR_RF_Asym  0.943520  22.194008  4.541645e+00      7.312167  0.945113   

fold                                                       2             \
metric          QNMPIW           Std training_time      PICP     QNMPIW   
Names                                                                     
QR_GBR        6.247460  2.128933e+01      1.536469  0.710722   6.244327   
Split-CP_RF  22.576547  2.921912e-14      0.741792  0.977777  23.411904   
CQR_RF_Sym   15.078585  2.927615e+01     20.254049  0.997945  15.057791   
MAPIE_Naive  17.126996  2.030343e-14      0.020310  0.998868  17.415869   
MAPIE_CV+    18.358065  1.374734e+00      0.100012  0.999497  18.421355   
XBCF          0.616121  2.078661e+00     19.330109  0.445805   1.029508   
Local-CP_RF  16.309882  3.995472e+01    436.648083  0.988134  16.083975   
Split-CP_NN  16.531698  2.344269e-15     12.004693  0.993375  18.454447   
Local-CP_NN  10.665031  8.234448e+00     25.701387  0.981802  12.903019   
CQR_RF_Asym  21.602916  5.758334e+00      7.312167  0.943520  21.870262   

fold                                            3                           \
metric                Std training_time      PICP     QNMPIW           Std   
Names                                                                        
QR_GBR       2.089835e+01      1.574008  0.718730   6.212449  2.115838e+01   
Split-CP_RF  1.574386e-14      0.735969  0.985199  24.101604  2.155090e-14   
CQR_RF_Sym   2.361564e+01     20.286812  0.998197  15.368691  2.459192e+01   
MAPIE_Naive  2.884593e-14      0.020571  0.999413  17.633255  4.590897e-15   
MAPIE_CV+    1.283066e+00      0.104819  0.999748  18.529221  1.317621e+00   
XBCF         1.885650e+00     19.449708  0.393182   0.810494  2.362153e+00   
Local-CP_RF  3.934244e+01    432.131560  0.985115  16.312704  4.053000e+01   
Split-CP_NN  1.197471e-14     35.284338  0.990943  15.896361  1.264020e-14   
Local-CP_NN  1.540771e+01     49.650776  0.983396  13.136190  1.623570e+01   
CQR_RF_Asym  5.419489e+00      7.312167  0.942723  21.731894  5.328490e+00   

fold                              4                                         
metric      training_time      PICP     QNMPIW           Std training_time  
Names                                                                       
QR_GBR           1.571387  0.723385   6.229027  2.141134e+01      1.539364  
Split-CP_RF      0.731548  0.979244  22.059621  2.909496e-14      0.736127  
CQR_RF_Sym      19.863849  0.997778  14.697023  2.519317e+01     20.240557  
MAPIE_Naive      0.021805  0.999497  16.958248  1.989306e-14      0.019008  
MAPIE_CV+        0.099496  0.999958  18.149629  1.513168e+00      0.091577  
XBCF            20.488347  0.435700   1.028187  1.776898e+00     19.971568  
Local-CP_RF    429.132111  0.989811  15.926637  3.940070e+01    432.036760  
Split-CP_NN     26.562179  0.997107  17.774421  5.819894e-16     22.574757  
Local-CP_NN     51.571535  0.994759  12.512565  1.898825e+01     43.554077  
CQR_RF_Asym      7.312167  0.945490  21.798388  5.867040e+00      7.312167

In [78]:
# change naming for plots
column_names = ['QR_GBR', 'Split-CP_RF',
       'CQR_RF_Sym', 'MAPIE_Naive',
       'MAPIE_CV+', 'XBCF', 'Local-CP_RF',
       'Split-CP_NN', 'Local-CP_NN',
       'CQR_RF_Asym']

In [28]:
column_names = [ 'XBCF','QR_GBR', 'Split-CP_RF',
       'MAPIE_Naive']


In [31]:
numbers = [1,2,3,4,5,6,7,8,9,10]
import numpy as np
import matplotlib.pyplot as plt
for alpha in quantiles:
    df = pd.DataFrame(unc_eval_test_dataframe_folds[alpha].T)


    PICP_cols = [col for col in df.columns if 'PICP' in col]
    MPIW_cols = [col for col in df.columns if 'MPIW' in col]
    QNMPIW_cols = [col for col in df.columns if 'QNMPIW' in col]
    df['Names'] = column_names
    df.set_index('Names', inplace= True)

    # plot for average width
    df[MPIW_cols].T.boxplot(vert=False)
    plt.title("Average Width for Miscoverage of "+str( alpha*100)+"%" )
    plt.xlabel("MPIW")
    plt.ylabel("PI Models")
    plt.savefig(f"figures/{today}_{alpha}_uncertainty_evaluation_width_cv.pdf",bbox_inches='tight')
    plt.close()

    df[QNMPIW_cols].T.boxplot(vert=False)
    plt.title("Quantile-Normalized Average Width for Miscoverage of "+str( alpha*100)+"%" )
    plt.xlabel("QNMPIW")
    plt.ylabel("PI Models")
    plt.savefig(f"figures/{today}_{alpha}_uncertainty_evaluation_normalized_width_cv.pdf",bbox_inches='tight')
    plt.close()

    df[PICP_cols].T.boxplot(vert=False)
    plt.title("Coverage of PI Methods for Miscoverage level "+str( alpha*100)+"%" )
    plt.xlabel("PICP")
    plt.ylabel("PI Models")
    plt.axvline(x=1-alpha, color='r')
    plt.savefig(f"figures/{today}_{alpha}_uncertainty_evaluation_picp_cv.pdf",bbox_inches='tight')
    plt.close()


Look at percentiles of interval length:

In [24]:
percentiles = [helper.calc_percentiles(outcome_dict["prediction_intervals"])
             for outcome_dict in predictions_test]

In [59]:
# look into specific percentiles (verify std intuition)
percentiles

[{0.05: {'Agnostic_QR_two-model': {'width_percentiles':                   0
    count  23849.000000
    mean     129.852818
    std       31.894727
    min       17.103240
    25%      109.296913
    50%      128.948795
    75%      148.886214
    max      322.675054},
   'RF_two-model': {'width_percentiles':                   0
    count  2.384900e+04
    mean   3.431591e+02
    std    1.421544e-10
    min    3.431591e+02
    25%    3.431591e+02
    50%    3.431591e+02
    75%    3.431591e+02
    max    3.431591e+02},
   'CQR_two-model_rf_QuantileRegErrFun': {'width_percentiles':                   0
    count  23849.000000
    mean     320.151404
    std       48.212525
    min      265.048028
    25%      287.865544
    50%      307.556252
    75%      336.721466
    max     1176.678934},
   'MAPIE_two-model_hurdle_naive': {'width_percentiles':                   0
    count  2.384900e+04
    mean   3.215530e+02
    std    2.022969e-10
    min    3.215530e+02
    25%    3.215530e+02
 

## Analysis of Interval Distribution

In [25]:
df = pd.DataFrame(predictions_test[0]['prediction_intervals'])
PI_model_names = df.index
predictions_test[0]['prediction_intervals']

{0.05: defaultdict(dict,
             {'Agnostic_QR_two-model': {'quantile_model': {'pred_low': array([ -74.38477607,  -59.10819946,  -59.22539415, ...,  -47.65722557,
                       -134.86767454,  -64.83915489]),
                'pred_high': array([ 86.5811635 ,  65.93328438,  66.59447338, ...,  55.89560411,
                       109.12604657,  67.72006337])}},
              'RF_two-model': {'quantile_model': {'pred_low': array([-177.53284179, -163.35537895, -160.2707863 , ..., -177.53773344,
                       -170.99616914, -181.82031972]),
                'pred_high': array([165.62628527, 179.80374811, 182.88834075, ..., 165.62139362,
                       172.16295792, 161.33880734])}},
              'CQR_two-model_rf_QuantileRegErrFun': {'quantile_model': {'pred_low': array([-177.04990372, -145.7661337 , -140.43251923, ..., -152.69329804,
                       -191.30774833, -200.76109781]),
                'pred_high': array([155.4717008 , 163.74559722, 152.25557

This boxplot shows us the distribution of the bounadaries across all folds, per PI model, per alpha

In [30]:
##### Merge all folds
# Seperate Plots
ylim = [-500, 500]
for alpha in [0.05,0.32]:
    for PI_model in PI_model_names:
        print(PI_model)
        predictions_combined = pd.concat([pd.DataFrame(fold['prediction_intervals'][alpha][PI_model]['quantile_model']) for fold in predictions_test])
        print(predictions_combined)
        plt.boxplot(x=predictions_combined)
        plt.ylim(ylim)
        plt.xticks([1, 2], ['Lower', 'Upper'])
        plt.xlabel('Bounadries')
        plt.ylabel('Interval Estimates')
        plt.title(f'Prediction Interval Boundaries for {PI_model} at {alpha}')
        plt.savefig(f"figures/PI_Boxplot/y_lim/Final_{alpha}_{PI_model}_estimates.pdf",bbox_inches='tight')
        plt.close()


Agnostic_QR_two-model
        pred_low  pred_high
0     -74.384776  86.581163
1     -59.108199  65.933284
2     -59.225394  66.594473
3     -63.722512  70.822318
4     -33.482952  46.772957
...          ...        ...
23844 -78.144107  86.157128
23845 -52.037193  62.595508
23846 -77.073375  87.369965
23847 -74.160861  87.592714
23848 -52.517590  62.376077

[119245 rows x 2 columns]
RF_two-model
         pred_low   pred_high
0     -177.532842  165.626285
1     -163.355379  179.803748
2     -160.270786  182.888341
3     -192.843681  150.315446
4     -164.995571  178.163556
...           ...         ...
23844 -198.812382  166.840038
23845 -182.190311  183.462108
23846 -177.886383  187.766037
23847 -173.975558  191.676862
23848 -163.443546  202.208873

[119245 rows x 2 columns]
CQR_two-model_rf_QuantileRegErrFun
         pred_low   pred_high
0     -177.049904  155.471701
1     -145.766134  163.745597
2     -140.432519  152.255578
3     -182.574335  143.824564
4     -145.519236  155.809419


## Analysis with Correlation: Oracle?

In [ ]:
column_names_corr = ['QR_GBR', 'Split-CP_RF',
       'CQR_RF_Sym', 'MAPIE_Naive',
       'MAPIE_CV+', 'XBCF', 'Local-CP_RF',
       'Split-CP_NN', 'Local-CP_NN',
       'CQR_RF_Asym', 'ITE']


## a) Correlation of TE and bounds

In [23]:
boundary = "pred_low"
#boundary = "pred_high"

In [114]:
df = calculate_correlation_matrix_corr(predictions_test, boundary = boundary,
                                       tau_true=tau_response)

0.05
       Agnostic_QR_two-model  RF_two-model  \
0                 -74.384776   -177.532842   
1                 -59.108199   -163.355379   
2                 -59.225394   -160.270786   
3                 -63.722512   -192.843681   
4                 -33.482952   -164.995571   
...                      ...           ...   
23844             -99.102704   -155.764955   
23845             -67.348861   -177.876205   
23846             -47.657226   -177.537733   
23847            -134.867675   -170.996169   
23848             -64.839155   -181.820320   

       CQR_two-model_rf_QuantileRegErrFun  MAPIE_two-model_naive  \
0                             -177.049904            -166.405999   
1                             -145.766134            -163.184436   
2                             -140.432519            -160.916620   
3                             -182.574335            -165.130097   
4                             -145.519236            -157.373737   
...                               

In [116]:
alpha_list = [0.05,0.32]
corr_dict = {}
for alpha in alpha_list:
    corr_dict[alpha] = {}

In [117]:
for folds in range(0,5):
    for alpha in alpha_list:
        corr_dict[alpha][folds] =df[folds][alpha]
        #print(corr_dict)

Create corrplot for each alpha in alpha list

In [118]:
df0 = corr_dict[0.05][4]
columns = df0.columns

In [119]:
# https://stackoverflow.com/questions/57226054/seaborn-correlation-matrix-with-p-values-with-python
for alpha in alpha_list:
    df0 = corr_dict[alpha][0]
    df1 = corr_dict[alpha][1]
    df2 = corr_dict[alpha][2]
    df3 = corr_dict[alpha][3]
    df4 = corr_dict[alpha][4]

    mean_corr = np.mean([df0,df1,df2,df3,df4], axis=0)

    #Change the column and index names
    #%%
    #columns = df0.columns
    df= pd.DataFrame.from_dict(mean_corr)
    df.columns = columns
    df.set_index(columns, inplace=True)
    #%%

    #%%
    #corr = df_5.corr()
    mask = np.triu(df)
    plot_cor_matrix(df,mask,x_axis_labels=column_names_corr)
    #plt.show()
    plt.savefig(f"figures/{alpha}_correlation_plot_{boundary}.pdf",bbox_inches='tight')
    plt.close()

## b) Correlation: width of PI and deviations of CATE and ITE


In [29]:
df, deviations = calculate_correlation_matrix_corr_width(predictions_test, tau_true=tau_response,
                                             CATE_model='two-model_hurdle_gbt')


{0: {}}
Agnostic_QR_two-model
RF_two-model
CQR_two-model_rf_QuantileRegErrFun
MAPIE_two-model_naive
MAPIE_two-model_cv_plus
xbcf_outcome_xbcf
Local-CP_two-model_rf
CP_two-model_NN
Local-CP_two-model_NN
CQR_two-model_rf_AsymRegErrFunc
(23849, 10)
(23849,)
   Agnostic_QR_two-model  RF_two-model  CQR_two-model_rf_QuantileRegErrFun  \
0              80.482970    171.579564                          166.260802   
1              62.520742    171.579564                          154.755865   
2              62.909934    171.579564                          146.344048   
3              67.272415    171.579564                          163.199450   
4              40.127955    171.579564                          150.664327   

   MAPIE_two-model_naive  MAPIE_two-model_cv_plus  xbcf_outcome_xbcf  \
0             161.397568               165.894337           3.990371   
1             161.397568               165.647716           2.747398   
2             161.397568               164.901588           

In [30]:
alpha_list = [0.05,0.32]
corr_dict = {}
for alpha in alpha_list:
    corr_dict[alpha] = {}

In [31]:
for folds in range(0,5):
    print(folds)
    for alpha in alpha_list:
        corr_dict[alpha][folds] =df[folds][alpha]

0
1
2
3
4


Create corrplot for each alpha in alpha list

In [32]:
# https://stackoverflow.com/questions/57226054/seaborn-correlation-matrix-with-p-values-with-python
for alpha in alpha_list:
    print(alpha)
    df0 = corr_dict[alpha][0]
    df1 = corr_dict[alpha][1]
    df2 = corr_dict[alpha][2]
    df3 = corr_dict[alpha][3]
    df4 = corr_dict[alpha][4]

    mean_corr = np.mean([df0,df1,df2,df3,df4], axis=0)
    print(mean_corr)

    df= pd.DataFrame.from_dict(mean_corr)
    #
    columns = df0.columns
    df.columns = columns
    df.set_index(columns, inplace=True)
    #%%

    mask = np.triu(df)
    plot_cor_matrix(df,mask,x_axis_labels=column_names_corr)
    plt.savefig(f"figures/{alpha}_width_absolute_error_correlation_plot.pdf",bbox_inches='tight')
    plt.close()

0.05
[[ 1.00000000e+00 -2.78115398e-03  2.51810393e-01             nan
   4.77323210e-02  8.68054181e-02  3.78811096e-01  6.96254155e-03
   3.88771719e-01  1.44873343e-01  1.21797286e-01]
 [-2.78115398e-03  1.00000000e+00 -1.98718293e-02             nan
  -4.74939042e-03 -7.83606879e-04 -8.78166897e-03  9.48338176e-02
   3.18272819e-03 -4.90187899e-03 -1.46946884e-03]
 [ 2.51810393e-01 -1.98718293e-02  1.00000000e+00             nan
   1.41353575e-01  4.75868459e-01  5.43150384e-01  8.78128435e-03
   2.33994336e-01  3.01144979e-01  1.46969855e-01]
 [            nan             nan             nan             nan
              nan             nan             nan             nan
              nan             nan             nan]
 [ 4.77323210e-02 -4.74939042e-03  1.41353575e-01             nan
   1.00000000e+00  1.22389489e-01  1.16631785e-01  3.43794636e-03
   3.90574831e-02  7.55604656e-02  4.00381210e-02]
 [ 8.68054181e-02 -7.83606879e-04  4.75868459e-01             nan
   1.22389489e

## Prediction distribution analysis for CATE Estimates

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

In [ ]:
##### Evaluation of distribution of predicted treatment effects

axes_limits = {"treatment_spending":[-20,20,0,0.4],
               "treatment_basket_value":[-20,20,0,0.3],
               "treatment_conversion":[-0.15, 0.25, 0, 60]}

clip_limits = {"treatment_spending":[-50,50],
               "treatment_basket_value":[-50,50],
               "treatment_conversion":[-0.5, 0.5]}

In [ ]:
# plot densities for treatment effects
for treatment_level in ["treatment_spending","treatment_basket_value","treatment_conversion"]:
    for fold_index in range(len(predictions_test)):
        with PdfPages(f"figures/{treatment_level}_distribution_fold{fold_index}.pdf") as pdf:
            for model in predictions_test[fold_index][treatment_level].keys():
                if model not in ["oracle__", "ATE__"]:
                    plt.figure()
                    plt.title(model)
                    plt.xlabel("Model Estimate")
                    plt.ylabel("Kernel Density")
                    plt.axis(axes_limits[treatment_level]) #
                    try:
                        sns.kdeplot(predictions_test[fold_index][treatment_level]["oracle__"])
                    except:
                        sns.kdeplot(predictions_test[fold_index][treatment_level]["oracle"])
                    sns.kdeplot(predictions_test[fold_index][treatment_level][model])
                    pdf.savefig()
                    plt.close()

Look at range of treatment effect predictions

In [ ]:
for treatment_level in ["treatment_spending"]:
    predictions_combined = pd.concat([pd.DataFrame(fold[treatment_level]) for fold in predictions_test])
print(predictions_combined.min())
print(predictions_combined.max())

In [ ]:
##### Merge all folds
for treatment_level in ["treatment_spending","treatment_basket_value","treatment_conversion"]:
    predictions_combined = pd.concat([pd.DataFrame(fold[treatment_level]) for fold in predictions_test])
    for model in predictions_combined.columns.values:
        with PdfPages(f"figures/{treatment_level}_{model}_distribution_combined.pdf") as pdf:
            if model not in ["oracle__", "ATE__"]:
                plt.figure()
                plt.title(model)
                plt.xlabel("Model Estimate")
                plt.ylabel("Kernel Density")
                plt.axis(axes_limits[treatment_level]) #
                try:
                    sns.kdeplot(predictions_combined[["oracle__"]].values.flatten(), linestyle="--", color='grey')
                except:
                    sns.kdeplot(predictions_combined[["oracle"]].values.flatten(), linestyle="--", color='grey')
                #sns.kdeplot(tau_response.values.flatten(), linestyle="--", color='green')
                sns.kdeplot(predictions_combined[[model]].values.flatten(),
                            clip=clip_limits[treatment_level], color="blue")
                pdf.savefig()
                plt.close()


## Prediction distribution analysis

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns

In [ ]:
##### Evaluation of distribution of predicted treatment effects

axes_limits = {"treatment_spending": [-20, 20, 0, 0.4],
               "treatment_basket_value": [-20, 20, 0, 0.3],
               "treatment_conversion": [-0.15, 0.25, 0, 60]}

clip_limits = {"treatment_spending": [-50, 50],
               "treatment_basket_value": [-50, 50],
               "treatment_conversion": [-0.5, 0.5]}

In [ ]:
# plot densities for treatment effects
for treatment_level in ["treatment_spending", "treatment_basket_value", "treatment_conversion"]:
    for fold_index in range(len(predictions_test)):
        with PdfPages(f"figures/CATE/{treatment_level}_distribution_fold{fold_index}.pdf") as pdf:
            for model in predictions_test[fold_index][treatment_level].keys():
                if model not in ["oracle__", "ATE__"]:
                    plt.figure()
                    plt.title(model)
                    plt.xlabel("Model Estimate")
                    plt.ylabel("Kernel Density")
                    plt.axis(axes_limits[treatment_level])  #
                    try:
                        sns.kdeplot(predictions_test[fold_index][treatment_level]["oracle__"])
                    except:
                        sns.kdeplot(predictions_test[fold_index][treatment_level]["oracle"])
                    sns.kdeplot(predictions_test[fold_index][treatment_level][model])
                    pdf.savefig()
                    plt.close()

Look at range of treatment effect predictions

In [ ]:
for treatment_level in ["treatment_spending"]:
    predictions_combined = pd.concat([pd.DataFrame(fold[treatment_level]) for fold in predictions_test])
print(predictions_combined.min())
print(predictions_combined.max())

In [ ]:
##### Merge all folds
for treatment_level in ["treatment_spending", "treatment_basket_value", "treatment_conversion"]:
    predictions_combined = pd.concat([pd.DataFrame(fold[treatment_level]) for fold in predictions_test])

    with PdfPages(f"figures/CATE/{treatment_level}_distribution_combined.pdf") as pdf:
        for model in predictions_combined.columns.values:
            if model not in ["oracle__", "ATE__"]:
                plt.figure()
                plt.title(model)
                plt.xlabel("Model Estimate")
                plt.ylabel("Kernel Density")
                plt.axis(axes_limits[treatment_level])  #
                try:
                    sns.kdeplot(predictions_combined[["oracle__"]].values.flatten(), linestyle="--", color='grey')
                except:
                    sns.kdeplot(predictions_combined[["oracle"]].values.flatten(), linestyle="--", color='grey')
                #sns.kdeplot(tau_response.values.flatten(), linestyle="--", color='green')
                sns.kdeplot(predictions_combined[[model]].values.flatten(),
                            clip=clip_limits[treatment_level], color="blue")
                pdf.savefig()
                plt.close()

In [ ]:
##### Merge all folds
for treatment_level in ["treatment_spending", "treatment_basket_value", "treatment_conversion"]:
    predictions_combined = pd.concat([pd.DataFrame(fold[treatment_level]) for fold in predictions_test])

    with PdfPages(f"figures/CATE/{treatment_level}_scatter_combined.pdf") as pdf:
        for model in predictions_combined.columns.values:
            if model not in ["oracle__", "ATE__"]:
                plt.figure()
                plt.title(model)
                plt.xlabel("Model Estimate")
                plt.ylabel("ITE")
                try:
                    sns.kdeplot(x=predictions_combined[[model]].values.flatten(),
                                y=predictions_combined[["oracle__"]].values.flatten(),
                                clip=clip_limits[treatment_level])
                except:
                    sns.kdeplot(x=predictions_combined[[model]].values.flatten(),
                                y=predictions_combined[["oracle"]].values.flatten(), clip=clip_limits[treatment_level])
                #sns.kdeplot(tau_response.values.flatten(), linestyle="--", color='green')
                pdf.savefig()
                plt.close()



## Analyze the distribution of the PIs

In [ ]:
df = pd.DataFrame(predictions_test[0]['prediction_intervals'])
PI_model_names = df.index
predictions_test[0]['prediction_intervals']

In [ ]:
##### Merge all folds
# Seperate Plots
ylim = [-100, 100]
for alpha in [0.05, 0.32]:
    for PI_model in PI_model_names:
        print(PI_model)
        predictions_combined = pd.concat(
            [pd.DataFrame(fold['prediction_intervals'][alpha][PI_model]['quantile_model']) for fold in
             predictions_test])
        print(predictions_combined)
        plt.boxplot(x=predictions_combined)
        #plt.ylim(ylim)
        plt.xticks([1, 2], ['Lower', 'Upper'])
        plt.xlabel('Bounadries')
        plt.ylabel('Interval Estimates')
        plt.title(f'Combined Oracle Prediction Intervals for {PI_model} at {alpha}')
        plt.savefig(f"figures/PI_Boxplot/{today}_{alpha}_{PI_model}_estimates.pdf", bbox_inches='tight')
        plt.close()


In [ ]:
##### Merge all folds
# All in one PDF
Tot = 12
Cols = 3

# Compute Rows required

Rows = Tot // Cols
Rows += Tot % Cols

# Create a Position index
Position = range(1, Tot + 1)

# Create main figure
#with PdfPages(f"figures/PI_Boxplot/{today}_combined_oracle.pdf") as pdf:
fig = plt.figure(1)
ylim = [-100, 100]
for k in range(Tot):

    for alpha in [0.05, 0.32]:
        # Plotting all the subplots

        for PI_model in PI_model_names:
            ax = fig.add_subplot(Rows, Cols, Position[k])
            predictions_combined = pd.concat(
                [pd.DataFrame(fold['prediction_intervals'][alpha][PI_model]['quantile_model']) for fold in
                 predictions_test])
            ax.boxplot(x=predictions_combined)
            ax.set_ylim(ylim)
            ax.set_xticklabels(['Lower', 'Upper'], fontdict=None, minor=False)
            #ax.set_xticks(ticks=[1, 2], ,minor=False)
            ax.set_xlabel('Bounadries')
            ax.set_ylabel('Interval Estimates')
            ax.set_title(f'{PI_model} at {alpha}')

fig.savefig(f"figures/PI_Boxplot/{today}_oracle_combined.pdf", bbox_inches='tight')
plt.close(fig)
